In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy.stats import multivariate_normal as mvnorm
from scipy.stats import uniform, invgamma, bernoulli, poisson, norm
import statsmodels.api as sm
import pandas as pd
from src.preprocessing import prepare_data, prepare_data_no_standardizing, MAR_data_deletion
from src.model_code import Gibbs_MH
import pymc3 as pm
import arviz as az
from statsmodels.tsa.stattools import acf 
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
df = pd.read_csv('student-mat.csv', sep=",")
features = ["age", "sex", "failures", "higher", "Medu", 
            "absences", "G2", "G3"]
df = df[features]
df.head()

,age,sex,failures,higher,Medu,absences,G2,G3
0,18,F,0,yes,4,6,6,6
1,17,F,0,yes,1,4,5,6
2,15,F,3,yes,1,10,8,10
3,15,F,0,yes,4,2,14,15
4,16,F,0,yes,3,4,10,10


In [4]:
X_df, y_df = prepare_data_no_standardizing(df)
X_df.head()

,age,failures,Medu,absences,G2,sex_M,higher_yes,intercept
0,18,0,4,6,6,0,1,1
1,17,0,1,4,5,0,1,1
2,15,3,1,10,8,0,1,1
3,15,0,4,2,14,0,1,1
4,16,0,3,4,10,0,1,1


In [52]:
import statsmodels.api as sm

In [86]:
gamma_reg = sm.GLS( X_df['higher_yes'], X_df[['intercept', 'age']], family=sm.families.Binomial())
gamma_res =gamma_reg.fit()

In [90]:
gamma_res.params.values

array([ 1.5499191 , -0.03596938])

In [88]:
gamma_res.cov_params()

,intercept,age
intercept,0.020193,-0.001202
age,-0.001202,0.000072


In [78]:
mat = np.matrix(X_df[['age', 'intercept']])
(mat.T*mat).I

matrix([[ 0.00155874, -0.02602502],
        [-0.02602502,  0.43705063]])

In [91]:
X_df_missing = MAR_data_deletion(X_df, 0.05, 0.05, 'higher_yes', 'G2')
X_df_missing

,age,failures,Medu,absences,G2,sex_M,higher_yes,intercept
0,18,0,4,6,6.0,0,1.0,1
1,17,0,1,4,5.0,0,1.0,1
2,15,3,1,10,8.0,0,1.0,1
3,15,0,4,2,14.0,0,1.0,1
4,16,0,3,4,10.0,0,1.0,1
...,...,...,...,...,...,...,...,...
390,20,2,2,11,9.0,1,1.0,1
391,17,0,3,3,16.0,1,1.0,1
392,21,3,1,3,8.0,1,1.0,1
393,18,0,3,0,12.0,1,1.0,1


In [115]:
X = X_df_missing
y = y_df.to_numpy()
B = 100
n = len(y)
higher_yes_col = 6
absences_col = 4
age_col = 0
taus = [2.9, 0.9, 0.7, 1]# [3.5, 0.8, 0.7, 1]
thin = 10

In [116]:
# Burn in thinning example
l = np.array(list(range(10)))
l[5::2]

array([5, 7, 9])

In [117]:
(betas, sigmas2, higher_yes_sim, G2_sim, alphas, gamas, etas) = Gibbs_MH(X, y, B, n, thin)

100%|██████████| 1999/1999 [00:15<00:00, 129.96it/s]


In [ ]:
betas_df = pd.DataFrame(betas.T, columns=[f"beta_{i}" for i in X_df.columns])
higher_yes_df = pd.DataFrame(higher_yes_sim.T, columns=[f"missing_higher_yes_{i}" for i in range(0, higher_yes_sim.shape[0])])
absences_df = pd.DataFrame(absences_sim.T, columns=[f"missing_absences_{i}" for i in range(0, absences_sim.shape[0])])
rest_df = pd.DataFrame({"sigmas2": sigmas2.T,
              "alpha0": alphas0.T,
              "alphas1": alphas1.T,
              "gammas0": gammas0.T, 
              "gammas1": gammas1.T
             })
              
results = pd.concat([betas_df , higher_yes_df, absences_df, rest_df], axis=1) 
results.head()
results.to_csv("results/03results_not_thinned.csv")
              

In [ ]:
def MCMC_diagnostics(chain, param):
    plt.subplot(411)
    plt.plot(chain)
    plt.title(f'Trace Plot {param}')

    plt.subplot(412)
    plt.hist(chain, bins=60)
    plt.title(f'Histogram {param}')

    plt.subplot(413)
    gw_plot = pm.geweke(chain)
    plt.scatter(gw_plot[:,0],gw_plot[:,1])
    plt.axhline(-1.98, c='r')
    plt.axhline(1.98, c='r')
    
    plt.ylim(-2.5,2.5)
    plt.title(f'Geweke Plot Comparing first 10% and Slices of the Last 50% of Chain {param}')

    plt.subplot(414)
    acf_values = acf(chain)
    plt.scatter(range(0, len(acf_values)), acf_values)
    plt.title(f'ACF {param}')
    
    plt.tight_layout()
    plt.show()

In [ ]:
MCMC_diagnostics(alphas0, "alpha0") # need to thin alphas and gammas

In [ ]:
MCMC_diagnostics(alphas1, 'alpha1')

In [ ]:
MCMC_diagnostics(gammas0, "gamma0")

In [ ]:
MCMC_diagnostics(gammas1, "gamma1")

In [ ]:
MCMC_diagnostics(sigmas2, "sigma2")

In [ ]:
for i, beta in enumerate(betas):
    MCMC_diagnostics(beta, X_df.columns[i])

In [ ]:
# thin
betas_thin = betas[:, ::thin]
higher_yes_sim_thin = higher_yes_sim[:, ::thin]
absences_sim_thin = absences_sim[:, ::thin]
alphas0_thin = alphas0[::thin]
alphas1_thin = alphas1[::thin]
gammas0_thin = gammas0[::thin]
gammas1_thin = gammas1[::thin]
sigmas2_thin = sigmas2[::thin]

In [ ]:
MCMC_diagnostics(alphas0_thin, "alpha0")
MCMC_diagnostics(alphas1_thin, "alpha1")
MCMC_diagnostics(gammas0_thin, "gamma0")
MCMC_diagnostics(gammas1_thin, "gamma1")

In [ ]:
plt.subplot(121)
plt.hist(higher_yes_sim.flatten())
plt.title(f'Simulated missing absences')


plt.subplot(122)
plt.hist(X[:, higher_yes_col])
plt.title(f'Observed absences')

plt.show()

In [ ]:
plt.subplot(121)
plt.hist(absences_sim.flatten(), bins=30)
plt.title(f'Simulated missing higher_yes')

plt.subplot(122)
plt.hist(X[:, absences_col], bins=30)
plt.title(f'Observed higher_yes')
plt.show()



In [ ]:
betas_df_thinned = pd.DataFrame(betas_thin.T, columns=[f"beta_{i}" for i in X_df.columns])
higher_yes_df_thinned = pd.DataFrame(higher_yes_sim_thin.T, columns=[f"missing_higher_yes_{i}" for i in range(0, higher_yes_sim.shape[0])])
absences_df_thinned = pd.DataFrame(absences_sim_thin.T, columns=[f"missing_absences_{i}" for i in range(0, absences_sim.shape[0])])
rest_df_thinned = pd.DataFrame({"sigmas2": sigmas2_thin.T,
              "alpha0": alphas0_thin.T,
              "alphas1": alphas1_thin.T,
              "gammas0": gammas0_thin.T, 
              "gammas1": gammas1_thin.T
             })
              
results_thinned = pd.concat([betas_df_thinned, higher_yes_df_thinned, absences_df_thinned, rest_df_thinned], axis=1) 
results_thinned.to_csv("results/03results_thinned.csv")
              

In [ ]:
from scipy.stats import norm

In [ ]:
norm.rvs(np.array([0,1,2]), 1)